In [101]:
import json
import os
import argparse
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from models import EncoderRNN, DecoderRNN, S2VTAttModel, S2VTModel
from dataloader import VideoDataset
import misc.utils as utils
from misc.cocoeval import suppress_stdout_stderr, COCOScorer
import numpy as np

from pandas.io.json import json_normalize
import pandas as pd

In [2]:
args = {'recover_opt': 'data/save/opt_info.json', 'saved_model': 'data/save/model_50.pth', 'dump_json': 1, 'results_path': 'results/', 'dump_path': 0, 'gpu': '0', 'batch_size': 25, 'sample_max': 1, 'temperature': 1.0, 'beam_size': 1}


In [3]:
opt = json.load(open(args["recover_opt"]))
for k, v in args.items():
        opt[k] = v
os.environ['CUDA_VISIBLE_DEVICES'] = opt["gpu"]

In [4]:
dataset = VideoDataset(opt, "test")

vocab size is  16860
number of train videos:  6501
number of val videos:  500
number of test videos:  2999
load feats from ['data/feats/resnet152']
max sequence length in data is 28


In [5]:
opt["vocab_size"] = dataset.get_vocab_size()

In [6]:
opt["seq_length"] = dataset.max_len

In [7]:
opt["model"]

'S2VTAttModel'

In [8]:
encoder = EncoderRNN(opt["dim_vid"], opt["dim_hidden"], bidirectional=opt["bidirectional"],
                             input_dropout_p=opt["input_dropout_p"], rnn_dropout_p=opt["rnn_dropout_p"]);
decoder = DecoderRNN(opt["vocab_size"], opt["max_len"], opt["dim_hidden"], opt["dim_word"],
                             input_dropout_p=opt["input_dropout_p"],
                             rnn_dropout_p=opt["rnn_dropout_p"], bidirectional=opt["bidirectional"]);
model = S2VTAttModel(encoder, decoder).cuda()

/home/chongke/anaconda3/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [9]:
model.load_state_dict(torch.load(opt["saved_model"]))

<All keys matched successfully>

In [10]:
crit = utils.LanguageModelCriterion()

/home/chongke/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [11]:
model.encoder.rnn.bidirectional = bool(model.encoder.rnn.bidirectional)

In [12]:
vocab = dataset.get_vocab()

In [13]:
model.eval()

S2VTAttModel(
  (encoder): EncoderRNN(
    (vid2hid): Linear(in_features=2048, out_features=512, bias=True)
    (input_dropout): Dropout(p=0.2, inplace=False)
    (rnn): GRU(512, 512, batch_first=True, dropout=0.5)
  )
  (decoder): DecoderRNN(
    (input_dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(16860, 512)
    (attention): Attention(
      (linear1): Linear(in_features=1024, out_features=512, bias=True)
      (linear2): Linear(in_features=512, out_features=1, bias=False)
    )
    (rnn): GRU(1024, 512, batch_first=True, dropout=0.5)
    (out): Linear(in_features=512, out_features=16860, bias=True)
  )
)

In [24]:
# loader = DataLoader(dataset, batch_size=opt["batch_size"], shuffle=True)

In [68]:
loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [69]:
scorer = COCOScorer()

init COCO-EVAL scorer


In [70]:
gt_dataframe = json_normalize(
        json.load(open(opt["input_json"]))['sentences'])

In [71]:
def convert_data_to_coco_scorer_format(data_frame):
    gts = {}
    for row in zip(data_frame["caption"], data_frame["video_id"]):
        if row[1] in gts:
            gts[row[1]].append(
                {'image_id': row[1], 'cap_id': len(gts[row[1]]), 'caption': row[0]})
        else:
            gts[row[1]] = []
            gts[row[1]].append(
                {'image_id': row[1], 'cap_id': len(gts[row[1]]), 'caption': row[0]})
    return gts

In [72]:
gts = convert_data_to_coco_scorer_format(gt_dataframe)

In [86]:
data = next(iter(loader))

In [87]:
data['labels'].shape

torch.Size([1, 28])

In [88]:
sents_label = [' '.join([e for e in [vocab.get(str(key.data.tolist())) for key in data['labels'][ind]] if e not in ('<eos>', '<sos>')]) for ind in range(data['labels'].shape[0])] 

In [99]:
opt["feats_dir"]

['data/feats/resnet152']

In [123]:
ix = 1
fc_feat = []
fc_feat.append(np.load(os.path.join(opt["feats_dir"][0], 'video%i.npy' % (ix))))
fc_feat = np.concatenate(fc_feat, axis=1)
fc_feat = fc_feat[None, :, :]

In [124]:
fc_feat=torch.from_numpy(fc_feat).type(torch.FloatTensor)

In [125]:
fc_feat.shape

torch.Size([1, 40, 2048])

In [126]:
fc_feat = fc_feat.cuda()

In [127]:
# forward the model to also get generated samples for each image
with torch.no_grad():
    seq_probs, seq_preds = model(
        fc_feat, mode='inference', opt=opt)

/home/chongke/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1339: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [128]:
sents = utils.decode_sequence(vocab, seq_preds)

In [129]:
sample_result = [video_ids, sents, sents_label]

In [130]:
pd.options.display.max_colwidth = 150
df_result = pd.DataFrame(sample_result).T

In [131]:
df_result.columns = ['id', 'inference', 'label']

In [132]:

df_result

,id,inference,label
0,video7114,a man is cooking,a man and a woman talking to each other


In [96]:
gt_dataframe.loc[gt_dataframe.index[gt_dataframe['video_id'] == video_ids[0]]]

,caption,sen_id,video_id
59900,a girl and a man are talking to each other,59900,video7114
59901,a girl helps a man detective prevent crimes,59901,video7114
59902,a man and a woman talking to each other,59902,video7114
59903,a man and woman are seriously discussing about something,59903,video7114
59904,a man speaks to a woman in a doorway,59904,video7114
59905,a man stands outside a glass door,59905,video7114
59906,a movie trailer about a man investigating a murder,59906,video7114
59907,a narrator talks about a show s plot,59907,video7114
59908,a show is voiced over,59908,video7114
59909,a thriller movie trailer,59909,video7114
